In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import torch.nn.functional as F
import random

In [2]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [3]:
df_train = pd.read_csv('/kaggle/input/train.csv')
df_test = pd.read_csv('/kaggle/input/test.csv')

In [4]:
df_train['Category'].unique()

array(['Men Tshirts', 'Sarees', 'Kurtis', 'Women Tshirts',
       'Women Tops & Tunics'], dtype=object)

In [5]:
df_c1 = df_train[df_train['Category'] == 'Men Tshirts']
df_c2 = df_train[df_train['Category'] == 'Sarees']
df_c3 = df_train[df_train['Category'] == 'Kurtis']
df_c4 = df_train[df_train['Category'] == 'Women Tshirts']
df_c5 = df_train[df_train['Category'] == 'Women Tops & Tunics']

In [6]:
df_c1.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN


In [7]:
class LabelEncoderDict:
    def __init__(self):
        self.encoders = {}
        
    def fit(self, df, columns):
        """Fit label encoders for each column"""
        for col in columns:
            le = LabelEncoder()
            # Include NaN as a unique label by appending it to valid labels
            valid_labels = df[col].dropna().unique().tolist()
            valid_labels.append('NaN')  # Assign a label for NaN
            le.fit(valid_labels)
            self.encoders[col] = le
            
    def transform(self, df, columns):
        """Transform labels using fitted encoders"""
        encoded = np.zeros((len(df), len(columns)))
        for i, col in enumerate(columns):
            series = df[col].copy()
            # Replace NaNs with the string 'NaN' so they can be encoded
            series = series.fillna('NaN')
            encoded[:, i] = self.encoders[col].transform(series)
        return encoded
    
    def get_num_classes(self, column):
        """Get number of classes for a specific column"""
        return len(self.encoders[column].classes_)


class MultiLabelImageDataset(Dataset):
    def __init__(self, df, image_dir, transform_basic=None, transform_augmented=None, attr_columns=10,do_transform=True):
        self.df = df
        self.image_dir = image_dir
        self.transform_basic = transform_basic  # Basic transform without augmentation
        self.transform_augmented = transform_augmented  # Augmented transform with augmentation
        self.attr_columns = attr_columns
        self.do_transform = do_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get image path
        img_name = str(self.df.iloc[idx]['id']).zfill(6)
        img_path = os.path.join(self.image_dir, f"{img_name}.jpg")

        # Load image
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            image = Image.new('RGB', (512,512))

        # Apply random probability to choose between augmentation or notF
        if (random.random() > 0.5) and self.do_transform:
            if self.transform_basic:
                image = self.transform_basic(image)
        else:
            if self.transform_augmented:
                image = self.transform_augmented(image)

        # Ensure labels are integers and convert to tensor
        labels = torch.tensor(self.df.iloc[idx][self.attr_columns].astype(int).values, dtype=torch.long)

        return image, labels


def prepare_data(df, image_dir, batch_size=32, test_size=0.2, num_attr_columns=10):
    """
    Prepare data loaders and label encoders
    """
    # Define attribute columns
    attr_columns = [f'attr_{i}' for i in range(1, num_attr_columns+1)] # TODO: Changes with the number of column to consider
    
    # Create and fit label encoders
    label_encoders = LabelEncoderDict()
    label_encoders.fit(df, attr_columns)
    
    # Transform labels
    encoded_labels = label_encoders.transform(df, attr_columns)
    df_encoded = df.copy()
    for i, col in enumerate(attr_columns):
        df_encoded[col] = encoded_labels[:, i]
    
    # Split data
    train_df, val_df = train_test_split(df_encoded, test_size=test_size, random_state=42)
    
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((512,512)),  # TODO: Changes with (512, 512)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    transform_augmented = transforms.Compose([
    transforms.Resize((512,512)), 
    transforms.RandomRotation(degrees=15),  # Rotate by up to 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.RandomResizedCrop(size=(512, 512), scale=(0.8, 1.0)),  # Randomly crop and resize
    transforms.RandomPerspective(distortion_scale=0.1, p=0.5),  # Apply perspective distortion
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # Affine transformations
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),  # Apply random Gaussian blur
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet statistics
    ])
    # Create datasets
    train_dataset = MultiLabelImageDataset(
        train_df,
        image_dir,
        transform_basic=transform,
        transform_augmented = transform_augmented,
        attr_columns=attr_columns
    )
    
    val_dataset = MultiLabelImageDataset(
        val_df,
        image_dir,
        transform_basic=transform,
        transform_augmented = transform_augmented,
        attr_columns=attr_columns,
        do_transform = False
    )
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Get number of classes for each attribute
    num_classes_per_attr = [label_encoders.get_num_classes(col) for col in attr_columns]
    
    return train_loader, val_loader, label_encoders, num_classes_per_attr

class MultiLabelCELoss(nn.Module):
    def __init__(self):
        super(MultiLabelCELoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.05)

    def forward(self, outputs, targets):
        # outputs is a list of predictions for each label
        # targets is a tensor of shape (batch_size, num_labels)
        loss = 0
        for i, output in enumerate(outputs):
            loss += self.criterion(output, targets[:, i])
        return loss / len(outputs)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from tqdm import tqdm 
from transformers import ConvNextModel

class MultiLabelClassifier(nn.Module):
    def __init__(self, num_classes_per_attr):
        super(MultiLabelClassifier, self).__init__()
        
        # Use ConvNeXt-Base with unfrozen backbone
        self.backbone = ConvNextModel.from_pretrained("facebook/convnext-base-384-22k-1k")
        backbone_features = self.backbone.config.hidden_sizes[-1]  # 1024 for base model
        
        # Modified feature processing without fixed dimensions
        self.feature_processor = nn.Sequential(
            nn.Conv2d(backbone_features, 1024, kernel_size=1),
            nn.GELU(),
            nn.Dropout(0.1)
        )
        
        # Complex classifier heads for each attribute
        self.classifier_heads = nn.ModuleList()
        for num_classes in num_classes_per_attr:
            classifier_head = nn.Sequential(
                # First branch - Spatial attention
                nn.Sequential(
                    nn.Conv2d(1024, 512, kernel_size=3, padding=1, groups=32),
                    nn.GELU(),
                    nn.Conv2d(512, 512, kernel_size=3, padding=1, groups=32),
                    nn.GELU(),
                ),
                
                # Second branch - Channel attention (SE-like module)
                nn.Sequential(
                    nn.AdaptiveAvgPool2d(1),
                    nn.Flatten(),
                    nn.Linear(512, 128),
                    nn.GELU(),
                    nn.Linear(128, 512),
                    nn.Sigmoid(),
                ),
                
                # Combine branches and final classification
                nn.Sequential(
                    nn.AdaptiveAvgPool2d(1),
                    nn.Flatten(),
                    nn.Linear(512, 1024),
                    nn.LayerNorm(1024),
                    nn.GELU(),
                    nn.Dropout(0.2),
                    nn.Linear(1024, 512),
                    nn.LayerNorm(512),
                    nn.Sigmoid(),
                    nn.Dropout(0.1),
                    nn.Linear(512, num_classes)
                )
            )
            self.classifier_heads.append(classifier_head)
    
    def forward(self, x):
        # Extract features from ConvNeXt backbone
        features = self.backbone(x).last_hidden_state
        
        # Process features
        processed_features = self.feature_processor(features)
        
        outputs = []
        for classifier_head in self.classifier_heads:
            # Spatial attention branch
            spatial_features = classifier_head[0](processed_features)
            
            # Channel attention branch
            channel_attention = classifier_head[1](spatial_features)
            channel_attention = channel_attention.view(-1, 512, 1, 1)
            
            # Apply channel attention and get final output
            attended_features = spatial_features * channel_attention
            output = classifier_head[2](attended_features)
            outputs.append(output)
            
        return outputs



class MultiLabelCELoss(nn.Module):
    def __init__(self):
        super(MultiLabelCELoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        loss = 0
        for i, output in enumerate(outputs):
            loss += self.criterion(output, targets[:, i])
        return loss / len(outputs)

from tqdm import tqdm  # Import tqdm for progress tracking

from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
from tqdm import tqdm
import torch.nn as nn

def train_model(model, train_loader, val_loader, num_epochs, num_classes_per_attr, model_type):
    
    model = torch.nn.DataParallel(model)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = MultiLabelCELoss()  # Assuming you have this loss function
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)

    # Reduce learning rate on plateau
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)

    # Early stopping params
    early_stopping_patience = 4
    early_stopping_counter = 0
    best_val_avg_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct_predictions = [0] * len(num_classes_per_attr)
        total_predictions = 0
        overall_correct = 0
        
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", unit="batch") as t:
            for images, labels in t:
                images, labels = images.to(device), labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                
                train_loss += loss.item()
                t.set_postfix(loss=loss.item())
                
                all_labels_match = torch.ones(labels.size(0), dtype=torch.bool, device=device)
                for i, output in enumerate(outputs):
                    _, predicted = torch.max(output, 1)
                    correct_predictions[i] += (predicted == labels[:, i]).sum().item()
                    all_labels_match &= (predicted == labels[:, i])
                    
                overall_correct += all_labels_match.sum().item()
                total_predictions += labels.size(0)
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct_predictions = [0] * len(num_classes_per_attr)
        val_total_predictions = 0
        val_overall_correct = 0
        
        with tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", unit="batch") as v:
            with torch.no_grad():
                for images, labels in v:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    v.set_postfix(loss=loss.item())
                    
                    all_labels_match_val = torch.ones(labels.size(0), dtype=torch.bool, device=device)
                    for i, output in enumerate(outputs):
                        _, predicted = torch.max(output, 1)
                        val_correct_predictions[i] += (predicted == labels[:, i]).sum().item()
                        all_labels_match_val &= (predicted == labels[:, i])
                    
                    val_overall_correct += all_labels_match_val.sum().item()
                    val_total_predictions += labels.size(0)
        
        # Print results
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        avg_acc = 0
        for i in range(len(num_classes_per_attr)):
            train_acc = 100 * correct_predictions[i] / total_predictions
            val_acc = 100 * val_correct_predictions[i] / val_total_predictions
            avg_acc += val_acc
            print(f'Attribute {i+1} - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')
        avg_acc = avg_acc/(len(num_classes_per_attr))
        overall_train_acc = 100 * overall_correct / total_predictions
        overall_val_acc = 100 * val_overall_correct / val_total_predictions
        print(f'Overall Train Accuracy: {overall_train_acc:.2f}%')
        print(f'Overall Validation Accuracy: {overall_val_acc:.2f}%')
        
        # Early stopping logic based on validation overall accuracy
        if avg_acc >= best_val_avg_acc:
            best_val_avg_acc = avg_acc
            torch.save(model.module.state_dict(), f'best_model_{model_type}.pth')  # Save the best model
            early_stopping_counter = 0  # Reset early stopping counter
        else:
            early_stopping_counter += 1
        
        # ReduceLROnPlateau 
        lr_scheduler.step(overall_val_acc)
        
        # Check early stopping condition
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered")
    torch.save(model.module.state_dict(), f'best_model_end_{model_type}.pth')

In [9]:
import pickle
import gc

def main(df_c1, df_c2, df_c3, df_c4, df_c5):
    # Set image directory
    image_dir = '/kaggle/input/train_images'
    
    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Define a function to train each model sequentially
    def train_single_model(data, num_attr_columns, model_type):
        # Prepare data
        print(f"Preparing data for {model_type}")
        train_loader, val_loader, label_encoders, num_classes_per_attr = prepare_data(data, image_dir, batch_size=16, num_attr_columns=num_attr_columns)
        world_size = torch.cuda.device_count()
        # Initialize the model
        print(f"Initializing model {model_type}")
        model = MultiLabelClassifier(num_classes_per_attr).to(device)
        
        # Define loss and optimizer
        criterion = MultiLabelCELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Train the model
        print(f"Training model {model_type}")
        
        train_model(model, train_loader, val_loader, num_epochs=40, num_classes_per_attr=num_classes_per_attr, model_type=model_type)
        
        # Save label encoders
        with open(f'label_encoders_{model_type}.pkl', 'wb') as f:
            pickle.dump(label_encoders, f)

        print("---------------------------------------------")
        print(f"number of classes for {model_type} is {num_classes_per_attr}")
        print("---------------------------------------------")

        # Free up memory
        del model, train_loader, val_loader, label_encoders, num_classes_per_attr
        torch.cuda.empty_cache()
        gc.collect()
    
    # Train models one at a time
#     train_single_model(df_c1, num_attr_columns=5, model_type="c1")
#     train_single_model(df_c2, num_attr_columns=10, model_type="c2")
    train_single_model(df_c3, num_attr_columns=9, model_type="c3")
#     train_single_model(df_c4, num_attr_columns=8, model_type="c4")
#     train_single_model(df_c5, num_attr_columns=10, model_type="c5")

In [10]:
main(df_c1, df_c2, df_c3, df_c4, df_c5)

Preparing data for c3
Initializing model c3


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/354M [00:00<?, ?B/s]

Training model c3


Epoch 1/40 [Validation]: 100%|██████████| 86/86 [02:38<00:00,  1.84s/batch, loss=0.607]


...

Epoch 38/40
Training Loss: 0.1466
Validation Loss: 0.9581
Attribute 1 - Train Acc: 98.85%, Val Acc: 87.33%
Attribute 2 - Train Acc: 86.07%, Val Acc: 61.39%
Attribute 3 - Train Acc: 86.88%, Val Acc: 58.83%
Attribute 4 - Train Acc: 98.17%, Val Acc: 89.23%
Attribute 5 - Train Acc: 87.08%, Val Acc: 64.76%
Attribute 6 - Train Acc: 85.69%, Val Acc: 62.12%
Attribute 7 - Train Acc: 85.87%, Val Acc: 61.68%
Attribute 8 - Train Acc: 99.29%, Val Acc: 96.12%
Attribute 9 - Train Acc: 99.69%, Val Acc: 98.32%
Overall Train Accuracy: 62.64%
Overall Validation Accuracy: 23.81%
Early stopping triggered


Epoch 39/40 [Validation]: 100%|██████████| 86/86 [02:27<00:00,  1.72s/batch, loss=0.785]


Epoch 39/40
Training Loss: 0.1488
Validation Loss: 0.9601
Attribute 1 - Train Acc: 98.66%, Val Acc: 86.74%
Attribute 2 - Train Acc: 86.07%, Val Acc: 60.73%
Attribute 3 - Train Acc: 86.81%, Val Acc: 58.39%
Attribute 4 - Train Acc: 98.41%, Val Acc: 89.16%
Attribute 5 - Train Acc: 87.08%, Val Acc: 64.84%
Attribute 6 - Train Acc: 85.21%, Val Acc: 62.34%
Attribute 7 - Train Acc: 86.02%, Val Acc: 62.12%
Attribute 8 - Train Acc: 99.19%, Val Acc: 95.75%
Attribute 9 - Train Acc: 99.73%, Val Acc: 98.17%
Overall Train Accuracy: 62.86%
Overall Validation Accuracy: 22.86%
Early stopping triggered


Epoch 40/40 [Validation]: 100%|██████████| 86/86 [02:30<00:00,  1.75s/batch, loss=0.904]


Epoch 40/40
Training Loss: 0.1466
Validation Loss: 0.9635
Attribute 1 - Train Acc: 98.72%, Val Acc: 87.25%
Attribute 2 - Train Acc: 86.29%, Val Acc: 61.90%
Attribute 3 - Train Acc: 87.26%, Val Acc: 58.97%
Attribute 4 - Train Acc: 98.35%, Val Acc: 88.64%
Attribute 5 - Train Acc: 87.21%, Val Acc: 65.57%
Attribute 6 - Train Acc: 85.03%, Val Acc: 62.20%
Attribute 7 - Train Acc: 85.98%, Val Acc: 61.90%
Attribute 8 - Train Acc: 99.25%, Val Acc: 95.82%
Attribute 9 - Train Acc: 99.73%, Val Acc: 98.17%
Overall Train Accuracy: 62.91%
Overall Validation Accuracy: 24.25%
Early stopping triggered
---------------------------------------------
number of classes for c3 is [14, 3, 3, 3, 3, 3, 3, 4, 3]
---------------------------------------------


In [11]:
test_df_c3 = df_test[df_test['Category'] == 'Kurtis']
test_df_c3

,id,Category
10889,11155,Kurtis
10890,11156,Kurtis
10891,11157,Kurtis
10892,11158,Kurtis
10893,11159,Kurtis
...,...,...
13344,13610,Kurtis
13345,13611,Kurtis
13346,13612,Kurtis
13347,13613,Kurtis


In [12]:
import torch
from torchvision import transforms
from PIL import Image
import pickle
from tqdm import tqdm
import time

def load_model(model_path, num_classes_per_attr, device):
    # Initialize the model architecture and load the saved weights
    model = MultiLabelClassifier(num_classes_per_attr)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = torch.nn.DataParallel(model)
    model.to(device)
    model.eval()
    return model

def load_label_encoders(encoder_path):
    with open(encoder_path, 'rb') as f:
        encoders = pickle.load(f)
    return encoders

def preprocess_image(image_path, image_size=(512,512)):
    # Define image transformations (same as used during training)
    transform = transforms.Compose([
        transforms.Resize((512,512)),  # TODO: Change with (512, 512)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    
    # Open image and apply transformations
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

def inference(image, model, label_encoders):
    # Perform inference
    with torch.no_grad():
        outputs = model(image)

    # Decode predictions
    predicted_labels = []
    for i, output in enumerate(outputs):
        _, predicted = torch.max(output, 1)

        # Attribute name should match the encoder dictionary keys like 'attr_1', 'attr_2', etc.
        attr_name = f'attr_{i + 1}'
        if attr_name in label_encoders.encoders:
            decoded_label = label_encoders.encoders[attr_name].inverse_transform([predicted.item()])[0]
            predicted_labels.append(decoded_label)
        else:
            raise KeyError(f"Encoder for {attr_name} not found in the loaded label encoders.")
    
    return predicted_labels

# Load the model and encoders once
model_path = "best_model_c3.pth"
encoder_path = "/kaggle/working/label_encoders_c3.pkl"
num_classes_per_attr = [14, 3, 3, 3, 3, 3, 3, 4, 3]
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = load_model(model_path, num_classes_per_attr, device)
label_encoders = load_label_encoders(encoder_path)

image_dir = "/kaggle/input/test_images"
preds = {f'attr_{i}': [] for i in range(1, 10)}

t1 = time.time()

for val in tqdm(test_df_c3['id'], desc='Processing Images', total=len(test_df_c3)):
    image_path = f"{image_dir}/{str(val).zfill(6)}.jpg"
    image = preprocess_image(image_path).to(device)  # Preprocess and send image to device
    predictions = inference(image, model, label_encoders)  # Use the already loaded model and encoders
    for i in range(1, 10):
        preds[f'attr_{i}'].append(predictions[i-1])

print(f'Time taken to process images is {time.time() - t1} seconds, which is {len(test_df_c3) / (time.time() - t1)} images per second')

Processing Images: 100%|██████████| 2460/2460 [04:38<00:00,  8.82it/s]

Time taken to process images is 278.78722310066223 seconds, which is 8.823933238257949 images per second


In [13]:
for i in range(1,10):
    test_df_c3[f'attr_{i}'] = preds[f'attr_{i}']
test_df_c3    

,id,Category,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9
10889,11155,Kurtis,red,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular
10890,11156,Kurtis,purple,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular
10891,11157,Kurtis,black,NaN,NaN,daily,NaN,solid,solid,three-quarter sleeves,regular
10892,11158,Kurtis,multicolor,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular
10893,11159,Kurtis,multicolor,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular
...,...,...,...,...,...,...,...,...,...,...,...
13344,13610,Kurtis,blue,a-line,knee length,party,net,solid,solid,three-quarter sleeves,regular
13345,13611,Kurtis,blue,a-line,knee length,daily,net,solid,solid,three-quarter sleeves,regular
13346,13612,Kurtis,maroon,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular
13347,13613,Kurtis,red,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular


In [14]:
test_df_c3.to_csv('test_validation_df_c3.csv',index=False)

In [15]:
import torch
from torchvision import transforms
from PIL import Image
import pickle
from tqdm import tqdm
import time

def load_model(model_path, num_classes_per_attr, device):
    # Initialize the model architecture and load the saved weights
    model = MultiLabelClassifier(num_classes_per_attr)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = torch.nn.DataParallel(model)
    model.to(device)
    model.eval()
    return model

def load_label_encoders(encoder_path):
    with open(encoder_path, 'rb') as f:
        encoders = pickle.load(f)
    return encoders

def preprocess_image(image_path, image_size=(512,512)):
    # Define image transformations (same as used during training)
    transform = transforms.Compose([
        transforms.Resize((512,512)),  # TODO: Change with (512, 512)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    
    # Open image and apply transformations
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

def inference(image, model, label_encoders):
    # Perform inference
    with torch.no_grad():
        outputs = model(image)

    # Decode predictions
    predicted_labels = []
    for i, output in enumerate(outputs):
        _, predicted = torch.max(output, 1)

        # Attribute name should match the encoder dictionary keys like 'attr_1', 'attr_2', etc.
        attr_name = f'attr_{i + 1}'
        if attr_name in label_encoders.encoders:
            decoded_label = label_encoders.encoders[attr_name].inverse_transform([predicted.item()])[0]
            predicted_labels.append(decoded_label)
        else:
            raise KeyError(f"Encoder for {attr_name} not found in the loaded label encoders.")
    
    return predicted_labels

# Load the model and encoders once
model_path = "best_model_end_c3.pth"
encoder_path = "/kaggle/working/label_encoders_c3.pkl"
num_classes_per_attr = [14, 3, 3, 3, 3, 3, 3, 4, 3]
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = load_model(model_path, num_classes_per_attr, device)
label_encoders = load_label_encoders(encoder_path)

image_dir = "/kaggle/input/test_images"
preds = {f'attr_{i}': [] for i in range(1, 10)}

t1 = time.time()

for val in tqdm(test_df_c3['id'], desc='Processing Images', total=len(test_df_c3)):
    image_path = f"{image_dir}/{str(val).zfill(6)}.jpg"
    image = preprocess_image(image_path).to(device)  # Preprocess and send image to device
    predictions = inference(image, model, label_encoders)  # Use the already loaded model and encoders
    for i in range(1, 10):
        preds[f'attr_{i}'].append(predictions[i-1])

print(f'Time taken to process images is {time.time() - t1} seconds, which is {len(test_df_c3) / (time.time() - t1)} images per second')

Processing Images: 100%|██████████| 2460/2460 [04:15<00:00,  9.64it/s]

Time taken to process images is 255.0995557308197 seconds, which is 9.64329330058428 images per second


In [16]:
for i in range(1,10):
    test_df_c3[f'attr_{i}'] = preds[f'attr_{i}']
test_df_c3    

,id,Category,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9
10889,11155,Kurtis,red,a-line,NaN,daily,NaN,solid,NaN,three-quarter sleeves,regular
10890,11156,Kurtis,maroon,a-line,knee length,daily,net,solid,solid,three-quarter sleeves,regular
10891,11157,Kurtis,black,a-line,calf length,daily,NaN,NaN,NaN,three-quarter sleeves,regular
10892,11158,Kurtis,multicolor,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular
10893,11159,Kurtis,multicolor,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular
...,...,...,...,...,...,...,...,...,...,...,...
13344,13610,Kurtis,blue,NaN,NaN,daily,net,solid,solid,three-quarter sleeves,regular
13345,13611,Kurtis,blue,straight,knee length,daily,net,solid,solid,three-quarter sleeves,regular
13346,13612,Kurtis,maroon,NaN,NaN,daily,NaN,NaN,default,three-quarter sleeves,regular
13347,13613,Kurtis,red,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular


In [17]:
test_df_c3.to_csv('test_df_c3.csv',index=False)

In [18]:
# import pandas as pd
# import torch
# from torchvision import transforms
# from PIL import Image
# import pickle
# import csv
# import time

# final_time = 0
# # Map categories to corresponding attribute columns (e.g., c1, c2, c3)
# category_to_attributes = {
#     'Men Tshirts': 'c1',
#     'Sarees': 'c2',
#     'Kurtis': 'c3',
#     'Women Tshirts': 'c4',
#     'Women Tops & Tunics': 'c5',
# }

# # Dummy value for missing attributes
# DUMMY_VALUE = "dummy_value"

# def load_model(model_path, num_classes_per_attr, device):
#     model = MultiLabelClassifier(num_classes_per_attr)
#     model.load_state_dict(torch.load(model_path, map_location=device))
#     model.to(device)
#     model.eval()
#     return model

# def load_label_encoders(encoder_path):
#     with open(encoder_path, 'rb') as f:
#         encoders = pickle.load(f)
#     return encoders

# def preprocess_image(image_path, image_size=(384,384)):
#     transform = transforms.Compose([
#         transforms.Resize(image_size),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])
#     image = Image.open(image_path).convert('RGB')
#     image = transform(image)
#     image = image.unsqueeze(0)  # Add batch dimension
#     return image

# def inference(image_path, model, label_encoders, num_classes_per_attr, device='cuda' if torch.cuda.is_available() else 'cpu'):
#     image = preprocess_image(image_path).to(device)
#     with torch.no_grad():
#         outputs = model(image)

#     predicted_labels = []
#     for i, output in enumerate(outputs):
#         _, predicted = torch.max(output, 1)
#         attr_name = f'attr_{i+1}'
#         if attr_name in label_encoders.encoders:
#             decoded_label = label_encoders.encoders[attr_name].inverse_transform([predicted.item()])[0]
#             predicted_labels.append(decoded_label)
#         else:
#             predicted_labels.append(DUMMY_VALUE)
    
#     return predicted_labels

# def make_predictions_for_dataset(df, model_paths, encoder_paths, output_path, image_dir, num_classes_per_attr_list, device='cuda' if torch.cuda.is_available() else 'cpu'):
#     final_time = 0
#     all_models = []
#     all_label_encoders = []
    
#     # Load models and encoders for each attribute (c1 to c5)
#     for model_path, encoder_path, num_classes_per_attr in zip(model_paths, encoder_paths, num_classes_per_attr_list):
#         model = load_model(model_path, num_classes_per_attr, device)
#         label_encoders = load_label_encoders(encoder_path)
#         all_models.append(model)
#         all_label_encoders.append(label_encoders)
    
#     all_predictions = []
    
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc="Making predictions"):
#         category = row['Category']
#         attribute_key = category_to_attributes.get(category, None)
#         if attribute_key:
#             # Determine which model and encoder to use based on the category
#             attribute_idx = int(attribute_key[1]) - 1  # Example: 'c1' -> index 0, 'c2' -> index 1, etc.
#             model = all_models[attribute_idx]
#             label_encoders = all_label_encoders[attribute_idx]
#             num_classes_per_attr = num_classes_per_attr_list[attribute_idx]
#             image_path = f"{image_dir}/{str(row['id']).zfill(6)}.jpg"  
            
#             try:
#                 t1 = time.time()
#                 predictions = inference(image_path, model, label_encoders, num_classes_per_attr, device)
#                 final_time = final_time + time.time() -t1 
#             except Exception as e:
#                 print(f"Error with image {image_path}: {e}")
#                 predictions = [DUMMY_VALUE] * len(num_classes_per_attr)  # Fallback in case of error
#         else:
#             predictions = [DUMMY_VALUE] * len(num_classes_per_attr_list[0])  # Handle if category has no mapping
            
#         # Pad the predictions to ensure each row has 10 attributes
#         while len(predictions) < 10:
#             predictions.append(DUMMY_VALUE)
        
#         all_predictions.append([row['id']] + predictions)
#     # Save predictions to CSV
#     with open(output_path, 'w', newline='') as csvfile:
#         writer = csv.writer(csvfile)
#         header = ['id'] + [f'c{i+1}' for i in range(10)]  # Fixed number of attributes as 10
#         writer.writerow(header)
#         writer.writerows(all_predictions)
#     print(f"Predictions saved to {output_path}")
#     print(f"Total model predictions time in seconds {final_time} and images predicted per second {len(test_df)/final_time}")

# # Example usage
# dataset_path = '/kaggle/input/meesho/test.csv'  # Path to your dataset
# model_paths = ['/kaggle/working/best_model_c1.pth', '/kaggle/working/best_model_c2.pth', '/kaggle/working/best_model_c3.pth', '/kaggle/working/best_model_c4.pth', '/kaggle/working/best_model_c5.pth']
# encoder_paths = ['/kaggle/working/label_encoders_c1.pkl', '/kaggle/working/label_encoders_c2.pkl', '/kaggle/working/label_encoders_c3.pkl', '/kaggle/working/label_encoders_c4.pkl', '/kaggle/working/label_encoders_c5.pkl']
# output_path = '/kaggle/working/output_predictions.csv'  # Path to save predictions
# image_dir = '/kaggle/input/meesho/test_images'  # Directory containing images
# num_classes_per_attr_list = [
#     [5, 3, 3, 4, 3],  # For c1
#     [5, 7, 4, 9, 5, 4, 5, 6, 10, 3],  # For c2
#     [14, 3, 3, 3, 3, 3, 3, 4, 3],  # For c3
#     [8, 4, 4, 4, 7, 4, 3, 3],  # For c4
#     [13, 5, 3, 8, 3, 4, 7, 5, 5, 7],  # For c5
# ]


# test_df = pd.read_csv(dataset_path)

# make_predictions_for_dataset(test_df, model_paths, encoder_paths, output_path, image_dir, num_classes_per_attr_list)
